In [ ]:
from aocd import get_data
raw_data = get_data(day=12, year=2025)
shapes = raw_data.split("\n\n")
regions = shapes.pop()

## Part 1 - Backtracking with overlap checking

In [ ]:
def create_base_shape(shape):
    base_shape = set()
    map = shape.split(":\n")[-1].splitlines()
    
    for i, line in enumerate(map):
        for j, char in enumerate(line):
            if char == "#":
                base_shape.add((i, j))
    return base_shape

def normalize_coords(orig_set):
    min_i = 10
    min_j = 10
    norm_set = set()
    for (i, j) in orig_set:
        min_i = min(min_i, i)
        min_j = min(min_j, j)
    for (i, j) in orig_set:
        norm_set.add((i - min_i, j - min_j))
    return norm_set

def rotate_shape(orig_set):
    rotated_set = set()
    for (i, j) in orig_set:
        rotated_set.add((j, -i))
    return rotated_set

def flip_shape(orig_set):
    flipped_set = set()
    for (i, j) in orig_set:
        flipped_set.add((i, -j))
    return flipped_set

def place_shape(variant, pos, grid):
    r, c = pos
    width, height = grid
    shifted = set()
    for (i, j) in variant:
        new_r, new_c = i + r, j + c
        if new_r < 0 or new_r >= height or new_c < 0 or new_c >= width:
            return None
        shifted.add((new_r, new_c))
    return shifted

def backtracking(grid, pieces_to_place, shape_list, occupied, piece_idx):
    if piece_idx == len(pieces_to_place):
        return True
    width = grid[0]
    height = grid[1]
    current_idx = pieces_to_place[piece_idx]
    variants = shape_list[current_idx]
    for variant in variants:
        for i in range(height):
            for j in range(width):
                placement = place_shape(variant, (i, j), grid)
                if placement and not (placement & occupied):
                    occupied.update(placement)
                    if backtracking(grid, pieces_to_place, shape_list, occupied, piece_idx+1): 
                        return True
                    else:
                        occupied -= placement
    return False

## Parsing the shapes and creating all variants
shape_list = []
cell_counts = []
for shape in shapes:
    orig_shape = create_base_shape(shape)
    cell_counts.append(len(orig_shape))
    shape_variants = set()
    for rotation in range(4):
        if rotation == 0:
            rotated_shape = normalize_coords(orig_shape)
        else:
            rotated_shape = normalize_coords(rotate_shape(rotated_shape))
        flipped_shape = normalize_coords(flip_shape(rotated_shape))
        shape_variants.add(frozenset(rotated_shape))
        shape_variants.add(frozenset(flipped_shape))
    shape_list.append(shape_variants)

## Parsing the regions
region_details = []
for region in regions.splitlines():
    region_split = region.split(": ")
    dimensions = tuple(int(dim) for dim in region_split[0].split("x"))
    counts = [int(count) for count in region_split[1].split(" ")]
    region_details.append([dimensions, counts])

In [ ]:
# Not all regions are feasible, only keep those that are:
feasibles = []
for region in region_details:
    counts = region[1]
    width, height = region[0]
    feasibles.append(sum(count * cells for count, cells in zip(counts, cell_counts)) <= width * height)
sum(feasibles)

feasible_regions = [region for region, feasible in zip(region_details, feasibles) if feasible]

In [ ]:
# Turns out all feasible solutions work out, so len(feasible_regions) would be enough
success_count = 0
for idx, region in enumerate(feasible_regions):
    grid = region[0]
    counts = region[1]
    pieces_to_place = [i for i, count in enumerate(counts) for _ in range(count)]
    occupied = set()
    success_count += int(backtracking(grid=grid, pieces_to_place=pieces_to_place, shape_list=shape_list, occupied=occupied, piece_idx=0))
print(success_count)